# ChatStats - Your ChatGPT Year in Review

ChatStats provides insights and analysis on your conversations with ChatGPT. Similar to Spotify's Year in Review, ChatStats offers a comprehensive overview of your chat history, highlighting key statistics and trends. As the field of prompt engineering continues to grow, ChatStats can be a valuable tool for understanding the dynamics of your chat conversations and learning how to craft more sophisticated prompts.

## Features

1. **Conversation Count**: ChatStats provides the total number of conversations in your chat history, giving you an idea of the volume of interactions.

2. **Messages per Conversation**: This metric calculates the average number of messages exchanged per conversation, helping you understand the depth of engagement.

3. **Messages by Month**: ChatStats analyzes your chat history on a monthly basis, providing a breakdown of the number of messages sent each month. This allows you to identify patterns and trends over time.

4. **User Type Analysis**: ChatStats categorizes users based on their roles (e.g., participant, moderator, tool) and provides insights into the number of messages sent by each user type. This analysis helps you understand the distribution of communication within your chat conversations.

5. **Top Tools**: If your chat conversations involve various tools or platforms, ChatStats identifies the most active tools based on the number of messages sent. This information can be valuable for assessing the effectiveness and popularity of different communication channels.

## How to Use ChatStats

To generate your ChatStats report, follow these steps:

1. Import your chat history data in JSON format. You can get this from the [ChatGPT website](https://chat.openai.com/#settings): Account Settings --> Data Controls --> Export Data.

2. Execute the `summarize_messages_by_month()` function, passing the JSON file as an argument. This function calculates various metrics, including messages by month, total messages, total conversations, and average messages per conversation.

3. Execute the `extract_messages_by_user_type()` function, passing the JSON file as an argument. This function categorizes users based on their roles and provides the number of messages sent by each user type.

4. Execute the `extract_tool_messages()` function, passing the JSON file as an argument. This function identifies the most active tools based on the number of messages sent.

5. Visualize the results using bar plots, line charts, or any other suitable visualization technique. You can use libraries like Matplotlib, Seaborn, or Plotly for this purpose.

By following these steps, you can gain valuable insights into your chat conversations and make data-driven decisions to improve communication and collaboration.

Remember, ChatStats is a powerful tool that helps you understand the patterns and dynamics of your chat history. Use it wisely to unlock the full potential of your conversations!


In [ ]:
import json
from collections import defaultdict
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def count_conversations_by_month(filename):
    with open(filename, 'r') as file:
        data = json.load(file)

    # Ensure the file contains a list
    if not isinstance(data, list):
        print("JSON file does not contain a list")
        return

    # Initialize a dictionary to hold the count of conversations per month
    conversations_per_month = defaultdict(int)

    # Count the conversations by month
    for entry in data:
        timestamp = entry.get('create_time')  # replace 'created_time' with the actual key for the date field
        if timestamp:
            date = datetime.fromtimestamp(timestamp)
            month = date.strftime('%Y-%m')  # get the year and month
            conversations_per_month[month] += 1

    return conversations_per_month

# Call the function with your filename
conversations_by_month = count_conversations_by_month("conversations.json")

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(conversations_by_month.items()), columns=['Month', 'Conversations'])

# Sort the DataFrame by month
df = df.sort_values('Month')

# Set the color palette to viridis
sns.set_palette('viridis',n_colors=3)

# Create the plot
sns.barplot(x='Month', y='Conversations', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
import json

def parse_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)

    # Ensure the file contains a list
    if not isinstance(data, list):
        print("JSON file does not contain a list")
        return

    # Get the first 20 entries
    first_20_entries = data[:20]

    # Print the first 20 entries
    for i, entry in enumerate(first_20_entries, start=1):
        print(entry)

    # Save the first 20 entries to a new JSON file
    with open("results.json", 'w') as outfile:
        json.dump(first_20_entries, outfile)

# Call the function with your filename
parse_json("conversations.json")

In [ ]:
import json
import csv

def extract_conversation_titles(filename):
    with open(filename, 'r') as file:
        data = json.load(file)

    # Ensure the file contains a list
    if not isinstance(data, list):
        print("JSON file does not contain a list")
        return

    # Initialize a list to hold the titles
    titles = []

    # Extract the titles and create_time
    for entry in data:
        title = entry.get('title')  # replace 'title' with the actual key for the title field
        create_time = entry.get('create_time')  # replace 'create_time' with the actual key for the create_time field
        if title:
            titles.append({'title': title, 'create_time': create_time})

    return titles

# Call the function with your filename
titles = extract_conversation_titles("conversations.json")

# Save titles to a CSV file
csv_filename = "conversation_titles.csv"
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Create Time"])  # Write header
    writer.writerows([[title['title'], title['create_time']] for title in titles])  # Write titles

print(f"Conversation titles saved to {csv_filename}")

## Message Counts

The below function performs some basic data analysis on my chat history - # of conversations per month, # of messages per conversation, total # of chats, etc. 

In [ ]:
import json
from collections import defaultdict
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
def summarize_messages_by_month(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    messages_by_month = defaultdict(int)
    total_messages = 0
    total_conversations = len(data)  # Count the total number of conversations
    total_messages_per_conversation = 0

    for conversation in data:
        conversation_messages = 0
        for message in conversation['mapping'].values():
            if message['message'] and 'create_time' in message['message'] and message['message']['create_time'] is not None:
                timestamp = datetime.fromtimestamp(message['message']['create_time'])
                month = timestamp.strftime('%Y-%m')  # format as year-month
                messages_by_month[month] += 1
                total_messages += 1
                conversation_messages += 1
        total_messages_per_conversation += conversation_messages

    average_messages_per_conversation = total_messages_per_conversation / total_conversations

    return dict(messages_by_month), total_messages, total_conversations, average_messages_per_conversation

data, total_messages, total_conversations, average_messages_per_conversation = summarize_messages_by_month('conversations.json')

# Calculate rolling average
rolling_average = np.convolve(list(data.values()), np.ones(3)/3, mode='same')
rolling_average = rolling_average[::-1]  # Reverse the order of the rolling average values

# Plotting
months = list(data.keys())[::-1]  # Reverse the order of the months
message_counts = list(data.values())[::-1]  # Reverse the order of the message counts

average_messages = total_messages / len(months)

# Calculate moving average for average messages per conversation
moving_average = np.convolve(list(data.values()), np.ones(3)/3, mode='same')
moving_average = moving_average[::-1]  # Reverse the order of the moving average values

# Set the color palette to viridis
sns.set_palette("viridis")

# Create the figure and axes
fig, ax1 = plt.subplots(figsize=(10, 5))

# Plot Number of Messages
ax1.bar(months, message_counts)
ax1.set_xlabel('Month')
ax1.set_ylabel('Number of Messages')
ax1.set_title('Number of Messages by Month')
ax1.tick_params(axis='x', rotation=45)
ax1.legend()

# Create a third y-axis
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Move the third y-axis to the right
ax3.plot(months, moving_average, color='green')
ax3.set_ylabel('Moving Average Messages per Conversation')
ax3.tick_params(axis='y', colors='green')
ax3.spines['right'].set_color('green')
ax3.yaxis.label.set_color('green')
ax3.legend()

# Set the limits of the y-axis for average messages per conversation
ax3.set_ylim(0, max(moving_average) * 1.1)

# Add total and average labels
plt.text(0.02, 0.95, f'Total Messages: {total_messages}', transform=ax1.transAxes)
plt.text(0.02, 0.9, f'Total Conversations: {total_conversations}', transform=ax1.transAxes)
plt.text(0.02, 0.85, f'Average Messages: {average_messages:.0f}', transform=ax1.transAxes)
plt.text(0.02, 0.8, f'Average Messages per Conversation: {average_messages_per_conversation:.0f}', transform=ax1.transAxes)

plt.tight_layout()
plt.show()

In [ ]:
def count_messages_by_user_type_and_month(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    messages_by_user_type_and_month = defaultdict(lambda: defaultdict(int))

    for conversation in data:
        for message in conversation['mapping'].values():
            if message['message'] and 'role' in message['message']['author'] and 'create_time' in message['message']:
                user_type = message['message']['author']['role']
                timestamp = message['message']['create_time']
                if timestamp is not None:
                    month = pd.to_datetime(timestamp, unit='s').strftime('%Y-%m')
                    messages_by_user_type_and_month[user_type][month] += 1

    return messages_by_user_type_and_month

messages_by_user_type_and_month = count_messages_by_user_type_and_month('conversations.json')

# Convert the dictionary to a DataFrame
df = pd.DataFrame(messages_by_user_type_and_month)

# Sort the DataFrame by month
df = df.sort_index()

# Set the color palette to viridis with 3 colors
sns.set_palette("viridis", n_colors=4)

# Increase the figure width
plt.figure(figsize=(20, 6))

# Create the plot
df.plot(kind='bar', stacked=True)
plt.xlabel('Month')
plt.ylabel('Messages')
plt.title('Messages by User Type and Month')
plt.legend(title='User Type')
plt.show()


The below Python script reads a JSON file containing chat conversations, counts the number of messages sent by each tool (where the role is 'tool'), and then prints a ranked list of tools based on the number of messages sent. 

In [ ]:
import seaborn as sns
def extract_tool_messages(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    tool_messages = defaultdict(int)

    for conversation in data:
        for message in conversation['mapping'].values():
            if message['message'] and 'role' in message['message']['author'] and message['message']['author']['role'] == 'tool':
                tool_name = message['message']['author']['name']
                tool_messages[tool_name] += 1

    return tool_messages

tool_messages = extract_tool_messages('conversations.json')

# Rank order the number of messages by each tool
ranked_tool_messages = sorted(tool_messages.items(), key=lambda x: x[1], reverse=True)

# Print the ranked number of messages by each tool
for rank, (tool_name, count) in enumerate(ranked_tool_messages, start=1):
    print(f"{rank}: {tool_name}: {count}")

# Create a horizontal bar plot of the ranked tool messages
tool_names = [tool_name for tool_name, count in ranked_tool_messages]
message_counts = [count for tool_name, count in ranked_tool_messages]

plt.figure(figsize=(10, 10))  # Increase figure size
sns.barplot(x=message_counts, y=tool_names, palette="viridis")  # Use horizontal bar plot with viridis color palette
plt.ylabel('Tool Name')
plt.xlabel('Message Count')
plt.title('Number of Messages by Tool')
plt.tight_layout()
plt.show()